In [1]:
import requests
from requests.auth import HTTPBasicAuth
import yaml
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
######GET DATASET FROM RAVE######
print('GET DATASET FROM RAVE')
auth = 'auth_gmb.yaml'
with open(auth) as f:
    auth = yaml.load(f, Loader = yaml.FullLoader)
r = requests.get(auth['API'], auth = HTTPBasicAuth(auth['USERNAME'], auth['PASSWORD']))
data_set = r.content.decode("utf-8")
data = BeautifulSoup(data_set)

GET DATASET FROM RAVE


In [3]:
######TRANSFORM DATASET######
print('TRANSFORM DATASET')
data_dict = {}
for clinicaldata in data.odm:
    node_name = clinicaldata.subjectdata.studyeventdata.formdata['formoid']
    for itemdata in clinicaldata.subjectdata.studyeventdata.formdata.itemgroupdata:
        if node_name not in data_dict.keys():
            data_dict[node_name] = {}
        if itemdata['itemoid'] not in data_dict[node_name].keys():
            data_dict[node_name][itemdata['itemoid']] = []
        try:
            data_dict[node_name][itemdata['itemoid']].append(itemdata['value'])
        except:
            data_dict[node_name][itemdata['itemoid']].append(None)

TRANSFORM DATASET


In [4]:
######PRINT DATA FILES######
print('PRINT DATA FILES')
for node_type in data_dict:
    df = pd.DataFrame()
    for node_key in data_dict[node_type]:
        df[node_key] = data_dict[node_type][node_key]
    file_name = auth['OUTPUT_FOLDER'] + node_type + ".tsv"
    if not os.path.exists(auth['OUTPUT_FOLDER']):
        os.mkdir(auth['OUTPUT_FOLDER'])
    df.to_csv(file_name, sep = "\t", index = False)

PRINT DATA FILES


In [5]:
######VALIDATE DATA FILES######
print('VALIDATE DATA FILES')
with open(auth['NODE_FILE']) as f:
    model = yaml.load(f, Loader = yaml.FullLoader)
for node in model['Nodes']:
    if node not in data_dict.keys():
        print(f'Data node {node} is not in the dataset')

for node in model['Nodes']:
    if node in data_dict.keys():
        for prop in model['Nodes'][node]['Props']:
            prop_name = node + '.' +  prop
            if prop_name not in data_dict[node].keys():
                print(f'Property {prop_name} is not in the dataset')

VALIDATE DATA FILES
Data node SURVIVAL is not in the dataset
Data node SURGERY_SUPPLEMENT is not in the dataset
Data node OFF_STUDY is not in the dataset
Data node ADVERSE_EVENTS is not in the dataset
Data node FormOID is not in the dataset
Data node FILE is not in the dataset
Data node SITE is not in the dataset
Data node PI_SIGNOFF is not in the dataset
Data node RADIATION_SUPPLEMENT is not in the dataset
Data node PHYSICAL_EXAM_SCREENING is not in the dataset
Data node CLINICALTRIAL is not in the dataset
Property LABORATORY.CRSE_NUM is not in the dataset
Property LABORATORY.CRSE_DAY_NUM is not in the dataset
Property LABORATORY.COMMENTS_HDR is not in the dataset
Property PRIOR_THERAPY_SUPPLEMENT.TX_STT_DT is not in the dataset
Property PRIOR_THERAPY_SUPPLEMENT.STOP_DT is not in the dataset
Property PRIOR_THERAPY_SUPPLEMENT.COMMENTS_HDR is not in the dataset
Property CONCOMITANT_MEASURES_MEDICATIONS.CRSE_NUM is not in the dataset
Property CONCOMITANT_MEASURES_MEDICATIONS.CRSE_DAY_NUM